<a href="https://colab.research.google.com/github/Mohamed-ux-beep/temperature-prediction/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install wandb

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#import wandb
from torch.utils.data import TensorDataset, DataLoader
import math
from tqdm import tqdm
import random

**Using wandb to optimize the hyperparameter**

In [ ]:
#wandb.login(key="b32a025379eee48c2089066791e79b06405c0d93")

In [ ]:
# Initialize W&B
#wandb.init(project="experiment3", entity="mohamedabokahf70")

In [5]:
# Define the hyperparameters you want to optimize
#config = wandb.config
#config.learning_rate = 0.001
#config.hidden_size = 64
#config.num_layers = 2
#config.batch_size = 32

In [3]:
data = pd.read_csv("/content/train.csv", header=None)

In [4]:
# spliting the data to training and validation 80% training and 20% testing
stop = 0.8 * len(data)
stop = math.floor(stop)
train_data = data[0:stop]
valid_data = data[stop:len(data)]

In [5]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
0,3.1,-0.5,0.0,-0.5,-0.6,1.1,2.2,1.3,1.3,-0.4,...,3.9,0.6,1.4,8.5,4.0,3.7,-1.3,-4.1,-3.9,-3.8
1,-5.3,-4.0,-2.4,-2.2,-8.4,-5.5,-1.4,-4.5,-0.3,-0.7,...,0.9,-0.4,-1.5,1.6,1.8,-0.3,1.2,0.8,-2.1,-3.1
2,-0.3,-3.3,-0.7,2.4,3.3,3.1,-1.8,-8.4,0.8,-1.8,...,2.9,-2.3,-8.1,-7.4,-6.1,-8.6,-8.1,-4.9,-2.1,-5.4
3,-8.7,-9.2,-8.6,-7.7,-7.2,-1.6,0.7,1.3,2.4,0.8,...,-4.9,-5.4,-1.0,-3.3,-6.3,-6.9,-0.4,-4.3,-5.8,-6.4
4,8.9,7.7,5.9,8.4,6.5,3.3,1.8,0.5,-1.0,1.3,...,5.0,3.9,4.2,9.4,11.2,5.3,5.6,9.4,11.2,9.9


In [6]:
valid_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
14,-6.0,1.3,1.1,1.3,1.8,2.6,2.8,-1.0,-11.2,-12.9,...,-4.6,-0.7,-2.0,-0.9,-4.7,-1.0,-0.2,-0.7,-0.9,-5.4
15,2.6,5.4,3.1,-0.4,-3.5,-0.1,2.4,5.7,4.1,3.1,...,4.4,3.3,1.4,0.7,-0.6,-0.1,-0.3,-0.9,2.5,1.5
16,-2.2,-0.9,1.8,-0.4,-2.1,-2.6,-3.8,-4.6,-5.0,-4.2,...,2.9,6.5,6.2,3.8,-0.1,-1.7,-4.9,-7.5,-3.3,-0.2
17,-1.9,-3.2,2.4,-1.5,-0.7,3.3,2.5,0.6,-1.1,1.6,...,9.4,4.0,1.0,3.7,6.3,3.2,8.0,8.6,8.5,9.0


In [7]:
# train_data is the training dataframe
def get_rows(train_data):
  x = []
  for i in range(len(train_data)):
    x.append(train_data.iloc[i])
  print(f" === returned X with the length of {len(train_data)} ===")
  return x

In [8]:
# x is a list with the value of each row in the data frame, data is data frame with the temperature input & target
def map_temp(x, data):
  for i in range(len(x)):
    for j in range(len(x[i])-97+1):
      new_row = {'Input': list(x[i][j:j+90]) , 'target': list(x[i][j+90:j+97])}
      row = pd.DataFrame([new_row])
      data = pd.concat([data, row], ignore_index=True)
  print(f" === returned dataframe with the size of {len(x) * (len(x[0])-96)} === ")
  return data

In [9]:
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

**preparing the training data for the data loader**

In [32]:
seed_everything(42)

In [33]:
train_df = pd.DataFrame(columns=['Input', 'target'])

In [34]:
print(" === Getting the X for the training data === \n")
X_train = get_rows(train_data)

 === Getting the X for the training data === 

 === returned X with the length of 14 ===


In [35]:
tr_data = map_temp(X_train, train_df)

 === returned dataframe with the size of 49756 === 


**preparing the validation data for dataloader**

In [36]:
valid_df = pd.DataFrame(columns=['Input', 'target'])

In [37]:
print(" === Getting the X for the Validation data === \n")
X_valid = get_rows(valid_data)

 === Getting the X for the Validation data === 

 === returned X with the length of 4 ===


In [38]:
val_data = map_temp(X_valid, valid_df)

 === returned dataframe with the size of 14216 === 


**Normalization of training data**

In [39]:
tr_data.head()

,Input,target
0,"[3.1, -0.5, 0.0, -0.5, -0.6, 1.1, 2.2, 1.3, 1....","[0.0, -2.1, -0.8, -3.1, -0.8, -0.4, -1.2]"
1,"[-0.5, 0.0, -0.5, -0.6, 1.1, 2.2, 1.3, 1.3, -0...","[-2.1, -0.8, -3.1, -0.8, -0.4, -1.2, 2.1]"
2,"[0.0, -0.5, -0.6, 1.1, 2.2, 1.3, 1.3, -0.4, 0....","[-0.8, -3.1, -0.8, -0.4, -1.2, 2.1, 3.1]"
3,"[-0.5, -0.6, 1.1, 2.2, 1.3, 1.3, -0.4, 0.4, -0...","[-3.1, -0.8, -0.4, -1.2, 2.1, 3.1, 3.3]"
4,"[-0.6, 1.1, 2.2, 1.3, 1.3, -0.4, 0.4, -0.7, 2....","[-0.8, -0.4, -1.2, 2.1, 3.1, 3.3, 3.4]"


In [108]:
tr_data.shape

(49756, 2)

In [40]:
scaler = MinMaxScaler(feature_range=(0, 1))
tr_inputs_normalized = scaler.fit_transform(list(tr_data['Input']))
tr_target_normalized = scaler.fit_transform(list(tr_data['target']))

In [41]:
val_inputs_normalized = scaler.fit_transform(list(val_data['Input']))
val_target_normalized = scaler.fit_transform(list(val_data['target']))

In [42]:
# convert the data to pytorch tensors
tr_inputs_tensor = torch.from_numpy(np.array(tr_inputs_normalized)).unsqueeze(-1).float()
tr_target_tensor = torch.from_numpy(np.array(tr_target_normalized)).unsqueeze(-1).float()
val_inputs_tensor = torch.from_numpy(np.array(val_inputs_normalized)).unsqueeze(-1).float()
val_target_tensor = torch.from_numpy(np.array(val_target_normalized)).unsqueeze(-1).float()

In [43]:
val_inputs_tensor.shape

torch.Size([14216, 90, 1])

In [44]:
val_target_tensor.shape

torch.Size([14216, 7, 1])

In [45]:
# preparing the dataloader
t_dataset = TensorDataset(tr_inputs_tensor, tr_target_tensor)
v_dataset = TensorDataset(val_inputs_tensor, val_target_tensor)
train_loader = DataLoader(t_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(v_dataset, batch_size=128)

**Defining LSTM Model**

In [46]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size , output_size) # fully connected take the 90 from 64 batch and transform them to 7 values

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [47]:
# Instantiate the LSTM model
input_size = 1
hidden_size = 64
output_size = 7
num_layers = 2
model = LSTM(input_size, hidden_size, output_size, num_layers)

In [48]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [49]:
for i, batch in enumerate(train_loader):
  if i == 0:
    a,b = batch
    print(a.shape, b.shape)

torch.Size([128, 90, 1]) torch.Size([128, 7, 1])


In [75]:
#sweep_config = {
#    "method": "random",
#    "parameters": {
#        "learning_rate": {"values": [0.001, 0.01, 0.1]},
#        "hidden_size": {"values": [32, 64, 128]},
#        "num_layers": {"values": [1, 2, 3]},
#        "batch_size": {"values": [16, 32, 64]}
#    }
#}

In [76]:
# Run the hyperparameter sweep
#sweep_id = wandb.sweep(sweep_config, project="experiment3", entity="mohamedabokahf70")

In [50]:
def train_model():
    # Early Stopping
    best_val_loss = float('inf')
    patience_counter = 0
    patience_limit = 10

    # Train the model
    num_epochs = 100
    for epoch in range(num_epochs):
        for tra_batch in train_loader:
            X_train, y_train = tra_batch
            optimizer.zero_grad()
            output = model(X_train)
            output = output.view(1, -1)
            loss = criterion(output, y_train.view(1, -1))
            loss.backward()
            optimizer.step()



        # Validate the model
        model.eval()
        val_losses = []
        with torch.no_grad():
            for val_batch in valid_loader:
                X_val, y_val = val_batch
                val_output = model(X_val)
                val_loss = criterion(val_output, y_val.squeeze())
                val_losses.append(val_loss.item())

        avg_val_loss = sum(val_losses) / len(val_losses)

        # Log training and validation loss with W&B (outside the validation loop)
        #wandb.log({"Training Loss": loss.item(), "Validation Loss": sum(val_losses) / len(val_losses)})
        print(f'Epoch [{epoch}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {avg_val_loss:.4f}')

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0  # Reset counter when new best loss is found
        else:
            patience_counter += 1
            if patience_counter >= patience_limit:
                print("Early stopping due to validation loss not improving")
                break


In [51]:
train_model()

Epoch [0/100], Training Loss: 0.0094, Validation Loss: 0.0106
Epoch [1/100], Training Loss: 0.0053, Validation Loss: 0.0072
Epoch [2/100], Training Loss: 0.0055, Validation Loss: 0.0069
Epoch [3/100], Training Loss: 0.0056, Validation Loss: 0.0068
Epoch [4/100], Training Loss: 0.0057, Validation Loss: 0.0067
Epoch [5/100], Training Loss: 0.0061, Validation Loss: 0.0066
Epoch [6/100], Training Loss: 0.0042, Validation Loss: 0.0066
Epoch [7/100], Training Loss: 0.0043, Validation Loss: 0.0065
Epoch [8/100], Training Loss: 0.0046, Validation Loss: 0.0064
Epoch [9/100], Training Loss: 0.0045, Validation Loss: 0.0062
Epoch [10/100], Training Loss: 0.0046, Validation Loss: 0.0061
Epoch [11/100], Training Loss: 0.0035, Validation Loss: 0.0058
Epoch [12/100], Training Loss: 0.0041, Validation Loss: 0.0057
Epoch [13/100], Training Loss: 0.0041, Validation Loss: 0.0057
Epoch [14/100], Training Loss: 0.0041, Validation Loss: 0.0056
Epoch [15/100], Training Loss: 0.0039, Validation Loss: 0.0055
Ep

In [48]:
#wandb.agent(sweep_id, function=train_model)

In [31]:
print(model)

LSTM(
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=7, bias=True)
)


**Reading the evaluation data**

In [52]:
test_x = pd.read_csv("/content/test_x.csv", header=None)

In [53]:
test_x.shape

(106, 90)

In [59]:
# data preparation
tst_x = []
for i in tqdm(range(len(test_x))):
  tst_x.append(list(test_x.iloc[i]))


100%|██████████| 106/106 [00:00<00:00, 11297.07it/s]


In [101]:
# normalization
test_x_normalized = scaler.fit_transform(tst_x)

In [102]:
# convert to tensors
test_x_tensors = torch.from_numpy(np.array(test_x_normalized)).unsqueeze(-1).float()

In [103]:
test_x_tensors.shape

torch.Size([106, 90, 1])

In [205]:
pred_y = model(test_x_tensors)

In [206]:
# transform the pred_y to array
pred_y_array = pred_y.detach().numpy()
type(pred_y_array)

numpy.ndarray

In [207]:
len(pred_y_array)

106

In [208]:
# i have to pad to the length 90 and then get only the first 7 values after denormalization
pred_y_array_padded = np.pad(pred_y_array,(0,83), 'constant', constant_values=(0, 0))

In [210]:
# we should denormalize the predictions
pred_normalized = scaler.inverse_transform(pred_y_array_padded)

In [215]:
pred_y_array_padded[106:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [213]:
pred_normalized[106:]

array([[-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999],
       [-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999],
       [-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999],
       ...,
       [-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999],
       [-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999],
       [-15.799999, -13.2     , -11.8     , ..., -11.4     , -17.9     ,
        -16.199999]], dtype=float32)

In [216]:
pred_normalized = pred_normalized[:106]

In [217]:
len(pred_normalized)

106

In [227]:
pred_normalized = [pred_normalized[i][:7].tolist() for i in range(len(pred_normalized))]

In [232]:
pred_df = pd.DataFrame(data=pred_normalized)

In [234]:
pred_df.to_csv('text_y.csv')

In [17]:
pred = pd.read_csv('/content/text_y.csv', index_col=0)

In [18]:
for i in range(len(pred)):
  pred.iloc[i] = round(pred.iloc[i], 1)

In [21]:
pred.to_csv('test_y.csv')

In [20]:
pred

,0,1,2,3,4,5,6
0,11.5,14.1,14.7,15.0,14.7,15.8,14.1
1,-0.4,1.8,2.6,3.4,2.2,1.9,1.0
2,1.9,3.9,4.6,5.4,4.3,4.2,3.1
3,6.0,8.4,9.0,9.5,8.7,9.1,7.6
4,4.1,6.5,7.1,7.8,7.0,7.2,6.1
...,...,...,...,...,...,...,...
101,14.5,17.0,17.0,17.2,17.0,18.6,16.8
102,3.7,5.2,5.5,5.9,4.8,4.4,3.3
103,6.1,8.4,8.8,9.3,8.4,8.6,7.3
104,18.9,20.1,19.9,19.5,19.2,20.7,18.6
